# Lista 2 - Exercício 1

Gere dois conjuntos de dados, $X$ (treinamento) e $X_1$ (teste), com $N=1000$ exemplos constituídos por vetores tridimensionais que pertencem a três classes equiprováveis $\omega_1$, $\omega_2$ e $\omega_3$. Os parâmetros das classes são:

$$
\begin{align*}
m_1 &= [0, 0, 0]^T \\
m_2 &= [1, 2, 2]^T & S_1 = S_2 = S_3 = \begin{bmatrix}0.8 & 0 & 0 \\ 0 & 0.8 & 0 \\ 0 & 0 & 0.8\end{bmatrix} = \sigma^2 I \\
m_3 &= [3, 3, 4]^T
\end{align*}
$$

a) Utilizando $X$, calcule o estimador ML da média e das matrizes de covariância para as três classes. Como assume-se que as três matrizes de covariância devem ser as mesmas, calcule para as três classes e realize uma média entre as três como a estimativa final.

In [1]:
import numpy as np
np.random.seed(42)
from scipy.spatial import distance

N = 1000

mu  = np.array([[0, 0, 0], [1, 2, 2], [3, 3, 4]])
cov = np.matrix([[0.8, 0, 0], [0, 0.8, 0], [0, 0, 0.8]])

train = np.array([np.random.multivariate_normal(m, cov, N) for m in mu])
test  = np.array([np.random.multivariate_normal(m, cov, N) for m in mu])

nclass = train.shape[0]

In [2]:
mu_ml = (1/N)*np.sum(train, axis=1)
print("Estimativa das médias:")
mu_ml

Estimativa das médias:


array([[ 0.04920336, -0.00533844,  0.04200233],
       [ 0.9331735 ,  1.95149438,  2.01271409],
       [ 2.98454873,  2.98521318,  4.03864776]])

In [3]:
cov_ml = np.mean([(1/N)*np.sum((train[i] - mu_ml[i]) ** 2, axis=0) for i in range(nclass)], axis=1) * np.identity(nclass)
print("Estimativa da matriz de covariância:")
cov_ml

Estimativa da matriz de covariância:


array([[0.77818638, 0.        , 0.        ],
       [0.        , 0.81275615, 0.        ],
       [0.        , 0.        , 0.8414038 ]])

b) Utilize o classificador com base em distância Euclideana para classificar os padrões de $X_1$, utilizando os valores computados (MLE) no item anterior.

In [4]:
labels_euclidean = []
for sample in np.vstack(test):
    # Norma de ordem 2 é equivalente à distância euclidiana
    distances = np.array([np.linalg.norm(sample - mu_ml[i], ord=2) for i in range(nclass)])
    labels_euclidean.append(np.argmin(distances))

c) Utilize o classificador com base em distância Mahalanobis para classificar os padrões de $X_1$, utilizando os valores computados (MLE) no item anterior.

In [5]:
labels_mahalanobis = []
for sample in np.vstack(test):
    distances = np.array([distance.mahalanobis(sample, mu_ml[i], np.linalg.inv(cov_ml)) for i in range(nclass)])
    labels_mahalanobis.append(np.argmin(distances))

d) Utilize o classificador de Bayes para classificar os padrões de $X_1$, utilizando os valores computados (MLE) no item anterior.

In [6]:
labels_bayes = []
for sample in np.vstack(test):
    pred = 0
    for i in range(1, nclass):
        mean_diff = mu_ml[pred] - mu_ml[i]
        if np.dot(mean_diff, sample) - 0.5 * (np.dot(mu_ml[pred], mu_ml[pred]) - np.dot(mu_ml[i], mu_ml[i])) < 0:
            pred = i
    labels_bayes.append(pred)

e) Para cada caso, calcule o erro de classificação e compare os resultados. Por que os desempenhos são próximos?

In [7]:
ground_truth = np.array([0] * N + [1] * N + [2] * N)
total = ground_truth.shape[0]

err_euclidean = np.count_nonzero(labels_euclidean != ground_truth) / total * 100
err_mahalanobis = np.count_nonzero(labels_mahalanobis != ground_truth) / total * 100
err_bayes = np.count_nonzero(labels_bayes != ground_truth) / total * 100

print("Erro de classificação\n")
print("Distância Euclideana:     {:.2f}%".format(err_euclidean))
print("Distância de Mahalanobis: {:.2f}%".format(err_mahalanobis))
print("Classificador de Bayes:   {:.2f}%".format(err_bayes))

Erro de classificação

Distância Euclideana:     5.90%
Distância de Mahalanobis: 5.80%
Classificador de Bayes:   5.90%


A acurácia dos classificadores baseados em distância são similares, pois a distância de Mahalanobis é equivalente à distância Euclideana para uma distribuição esférica, que é o caso dos dados gerados (a pequena diferença possivelmente se dá por conta do cálculo da inversa da matrix de covariância). Similarmente, o classificador de Bayes apresenta o mesmo desempenho, pois para distribuições esféricas (matriz de covariância diagonal), os classificadores baseados em distância são equivalentes ao de Bayes.